In [2]:
import pandas as pd
import re

In [3]:
file_name = 'chatgrup.csv'  # Ubah ini sesuai nama file Anda
data = pd.read_csv(file_name)

In [4]:
if 'Pesan' in data.columns:
    # Fungsi untuk membersihkan teks
    def bersihkan_teks(teks):
        if pd.isnull(teks):
            return ""
        # Hanya ambil angka, huruf, spasi, dan tanda baca umum
        return re.sub(r'[^a-zA-Z0-9\s.,!?-]', '', teks)
    
    # Terapkan pembersihan pada kolom 'Pesan'
    data['Pesan_Bersih'] = data['Pesan'].apply(bersihkan_teks)

    # Simpan hasilnya ke file baru
    cleaned_file_name = 'data_group_cleaned.csv'
    data.to_csv(cleaned_file_name, index=False)
    print(f"Data berhasil dibersihkan dan disimpan di {cleaned_file_name}")
else:
    print("Kolom 'Pesan' tidak ditemukan dalam file CSV. Pastikan format file benar.")

Kolom 'Pesan' tidak ditemukan dalam file CSV. Pastikan format file benar.


In [5]:
data.head()

08/04/23 07.31 - Pesan dan panggilan dienkripsi secara end-to-end. Tidak seorang pun di luar chat ini, termasuk WhatsApp, yang dapat membaca atau mendengarkannya. Ketuk untuk info selengkapnya.
0  24/04/23 08.57 - ‎+62 852-6441-5694 bergabung ...                                                                                                                                               
1  02/05/23 20.18 - +62 822-3849-2861: <Media tid...                                                                                                                                               
2  02/05/23 20.18 - +62 822-3849-2861: [COMFEASTS...                                                                                                                                               
3                                                NaN                                                                                                                                               
4              Communication Feast proudly present!✨

In [6]:
import re

# Rename the column to a simpler name for processing
data.columns = ['Raw_Text']

# Drop any empty rows
data = data.dropna(subset=['Raw_Text'])

# Define a function to parse the raw text into structured columns
def parse_message(row):
    match = re.match(r'^(\d{2}/\d{2}/\d{2}) (\d{2}:\d{2}) - (.+?): (.+)$', row)
    if match:
        return match.groups()  # Tanggal, Waktu, Pengirim, Pesan
    return (None, None, None, row)  # If it doesn't match, treat as a system message

# Apply parsing to the data
parsed_data = data['Raw_Text'].apply(parse_message)
parsed_df = pd.DataFrame(parsed_data.tolist(), columns=['Tanggal', 'Waktu', 'Pengirim', 'Pesan'])

# Display the structured data
parsed_df.head()

Tanggal Waktu Pengirim                                              Pesan
0    None  None     None  24/04/23 08.57 - ‎+62 852-6441-5694 bergabung ...
1    None  None     None  02/05/23 20.18 - +62 822-3849-2861: <Media tid...
2    None  None     None  02/05/23 20.18 - +62 822-3849-2861: [COMFEASTS...
3    None  None     None              Communication Feast proudly present!✨
4    None  None     None                          🎮E-Sport MLBB Tournament🎮

In [7]:
# Update the regex pattern to handle additional cases
def parse_message(row):
    # Pattern for normal messages
    normal_pattern = re.match(r'^(\d{2}/\d{2}/\d{2}) (\d{2}:\d{2}) - ([^:]+): (.+)$', row)
    if normal_pattern:
        return normal_pattern.groups()  # Tanggal, Waktu, Pengirim, Pesan
    
    # Pattern for system messages or announcements (e.g., joined group)
    system_pattern = re.match(r'^(\d{2}/\d{2}/\d{2}) (\d{2}:\d{2}) - (.+)$', row)
    if system_pattern:
        return system_pattern.groups() + (None,)  # Add None for Pesan if it's a system message
    
    return (None, None, None, row)  # If no match, keep the row as raw text

# Apply the updated parsing logic
parsed_data = data['Raw_Text'].apply(parse_message)
parsed_df = pd.DataFrame(parsed_data.tolist(), columns=['Tanggal', 'Waktu', 'Pengirim', 'Pesan'])

# Clean up the data by dropping rows where both Tanggal and Waktu are missing
parsed_df = parsed_df.dropna(subset=['Tanggal', 'Waktu'])

# Display the cleaned and structured data
parsed_df.head()

Empty DataFrame
Columns: [Tanggal, Waktu, Pengirim, Pesan]
Index: []

In [8]:
# Display the first few rows of the raw text data to analyze its actual structure
data['Raw_Text'].head(10).tolist()


['24/04/23 08.57 - \u200e+62 852-6441-5694 bergabung menggunakan tautan undangan grup ini',
 '02/05/23 20.18 - +62 822-3849-2861: <Media tidak disertakan>',
 '02/05/23 20.18 - +62 822-3849-2861: [COMFEASTSPORT CHAMPIONSHIP]',
 'Communication Feast proudly present!✨',
 '🎮E-Sport MLBB Tournament🎮',
 'Match Day 1: Sunday, June 4th 2023',
 'Final Day (on stage): Wednesday, June 7th 2023',
 'Available for All tier/Rank-Skin On',
 '💸PRIZE POOL💸',
 '1 : Rp 1.000.000']